In [7]:
import pandas as pd
import numpy as np
import pyodbc
import warnings
from utils2 import PyroBayesianLogisticRegression, k_fold_cross_validation
from joblib import Parallel, delayed
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from pycaret.classification import *
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'


from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import LeaveOneOut, cross_val_score

from keras.models import Model
from keras.layers import Input, Dense, LeakyReLU, BatchNormalization
from keras.optimizers import Adam

from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold

warnings.simplefilter("ignore")

ModuleNotFoundError: No module named 'tensorflow.tsl'

In [ ]:
def get_data(player_id, pts_thresh):
    server = 'localhost\SQLEXPRESS'
    database = 'nba_game_data'

    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
    cursor = cnxn.cursor()

    # Define the stored procedure call
    stored_proc_call = "{CALL seasonPtsDataSet (?, ?)}"  # Replace with your stored procedure name

    # Execute the stored procedure
    cursor.execute(stored_proc_call, (pts_thresh, player_id))

    # Fetch the data
    data = cursor.fetchall()

    # Execute the stored procedure
    cursor.execute(stored_proc_call, (pts_thresh, player_id))

    # Fetch the data
    data = cursor.fetchall()

    # Get column names from cursor description
    columns = [column[0] for column in cursor.description]

    # Create a DataFrame
    df = pd.DataFrame.from_records(data, columns=columns)

    # Return Data
    return df.iloc[:-1]  

In [ ]:
def encode_features(X,pred):    
    # Number of features in the DataFrame
    N = X.shape[1]

    # Target number of dimensions
    M = round(len(X)*.15) 

    # Normalize the data
    scaler = MinMaxScaler()
    df_season_scaled = scaler.fit_transform(X)
    pred_scaled = scaler.fit_transform(pred)

    # Define the model architecture
    input_layer = Input(shape=(N,))
    # Encoder layer, with BatchNormalization and LeakyReLU
    encoded = Dense(M)(input_layer)
    encoded = BatchNormalization()(encoded)
    encoded = LeakyReLU(alpha=0.01)(encoded)  
    # Decoder layer, output size is N (to reconstruct the original data)
    decoded = Dense(N, activation='sigmoid')(encoded)

    # Autoencoder model
    autoencoder = Model(input_layer, decoded)
    autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

    # Train the autoencoder
    autoencoder.fit(df_season_scaled, df_season_scaled,
                    epochs=50,
                    batch_size=256,
                    shuffle=True,
                    validation_split=0.2,
                    verbose=False)

    # Extract the encoder part of the autoencoder
    encoder = Model(input_layer, encoded)

    # Encode the scaled data
    df_season_encoded = encoder.predict(df_season_scaled,verbose = False)

    # Create a new DataFrame for the encoded data
    df_season_reduced = pd.DataFrame(df_season_encoded, columns=[f'Feature{i+1}' for i in range(M)])

    # Debugging: Inspect Intermediate Outputs to check for all-zero columns
#     encoder_output_model = Model(input_layer, encoded)
#     encoder_outputs = encoder_output_model.predict(df_season_scaled, verbose = True)
    
    pred = encoder.predict(pred_scaled, verbose = False)
    
    return df_season_reduced, pred


In [ ]:
df = get_data(1628374,24.5)

In [ ]:
df = df.fillna(df.median())

In [ ]:
X = df.drop(columns=['PtsThresh'])
y = df['PtsThresh']

In [ ]:
X_en, junk = encode_features(X,X)

In [ ]:
X_en['PtsThresh'] = y

In [ ]:
folds_ = int(len(X_en)*.5)
folds_

In [ ]:
# Setup the environment and specify the number of folds for cross-validation
exp_clf = setup(data=X_en, target='PtsThresh', session_id=1234, fold=15)  # Increase folds here

# Compare models to find the best one based on default metrics
best_model = compare_models()

In [20]:
# Tune the best model using TPE optimization with Optuna
best_model = tune_model(best_model, search_algorithm='bayesian', n_iter=50, search_library='sklean', choose_better=True)
# Display the tuned best model
print(best_model)

ModuleNotFoundError: 
'tune_sklearn' is a soft dependency and not included in the pycaret installation. Please run: `pip install tune-sklearn ray[tune]` to install.
Alternately, you can install this by running `pip install pycaret[tuners]`

In [24]:
X_en.drop(columns=['PtsThresh'])
predictions = predict_model(model, data=X_en.drop(columns=['PtsThresh']))

NameError: name 'df_new' is not defined

In [27]:
predictions = predict_model(best_model, data=X_en.drop(columns=['PtsThresh']))
predictions['actual'] = y
predictions

,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,prediction_label,prediction_score,actual
0,0.270759,0.996973,0.320384,0.520368,-0.002515,-0.002754,-0.004252,0,0.9345,0
1,0.682985,0.832869,0.248745,1.463021,-0.006382,-0.009701,-0.000268,0,0.9642,0
2,0.851928,1.070348,0.440286,1.143659,-0.002454,-0.004107,-0.007365,0,0.9596,0
3,1.343338,1.010278,1.055181,1.035669,-0.000070,-0.000720,-0.013013,0,0.9564,0
4,1.449142,1.073285,0.488032,0.583266,0.167180,-0.001447,-0.006897,1,1.0000,1
5,0.709851,0.616822,0.467653,0.376221,-0.001082,-0.004610,-0.004961,0,0.9254,0
6,0.354999,-0.000340,0.332401,0.514578,-0.003310,-0.006434,-0.004368,0,0.9470,1
7,0.626483,0.912087,0.518471,1.051061,0.032742,-0.002768,-0.002774,1,0.9895,0
8,1.620342,0.758724,1.444981,0.457451,0.112311,-0.001036,-0.002590,1,1.0000,0
9,0.577709,0.042392,1.822886,1.132611,0.201942,-0.001958,1.409826,1,1.0000,1


In [8]:
# List of models to evaluate
models = [
    ('Logistic Regression', LogisticRegression(solver='liblinear')),
    ('Decision Tree', DecisionTreeClassifier()),
    ('Naive Bayes', GaussianNB()),
    ('KNN', KNeighborsClassifier(n_neighbors=3))
]

results = []
for name, model in models:
    scores = cross_val_score(model, X_en, y, cv=12, scoring='f1', n_jobs=-1)
    results.append((name, scores.mean(), scores.std()))

In [9]:
# Select the best model based on mean F1 score, break ties with standard deviation
best_model_name, best_score, _ = max(results, key=lambda x: (x[1], -x[2]))

best_model_name, best_score

('Pyro', 0.3444444444444445)

In [64]:
def preform_dt_parameter_tuning(X,y):
    # Expanded search space with additional parameters
    space = {
        'criterion': hp.choice('criterion', ['gini', 'entropy']),
        'splitter': hp.choice('splitter', ['best', 'random']),
        'max_depth': hp.choice('max_depth', range(1, 15)),
        'min_samples_split': hp.uniform('min_samples_split', 0.01, 0.2),
        'min_samples_leaf': hp.uniform('min_samples_leaf', 0.01, 0.2),
        'max_features': hp.choice('max_features', ['auto', 'sqrt', 'log2', None]),
        'class_weight': hp.choice('class_weight', [None, 'balanced']),
        'min_weight_fraction_leaf': hp.uniform('min_weight_fraction_leaf', 0, 0.5),
        'max_leaf_nodes': hp.choice('max_leaf_nodes', [None] + list(range(2, 100)))
    }

    # Objective function with LOO cross-validation
    def objective(space):
        classifier = DecisionTreeClassifier(
            criterion=space['criterion'],
            splitter=space['splitter'],
            max_depth=space['max_depth'],
            min_samples_split=int(space['min_samples_split'] * 100),
            min_samples_leaf=int(space['min_samples_leaf'] * 100),
            max_features=space['max_features'],
            class_weight=space['class_weight'],
            min_weight_fraction_leaf=space['min_weight_fraction_leaf'],
            max_leaf_nodes=space['max_leaf_nodes']
        )

        # Using Leave-One-Out cross-validation
        loo = LeaveOneOut()
        accuracy = cross_val_score(classifier, X, y, cv=loo, scoring='f1', n_jobs=-1).mean()

        # Return the loss and status
        return {'loss': -accuracy, 'status': STATUS_OK}

    # Run the optimization
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=200,
        trials=trials
    )

    # Assuming `best` is the dictionary of best parameters returned by `hyperopt`
    # Map `hp.choice` indices to their corresponding values for all parameters using `hp.choice`
    criterion_options = ['gini', 'entropy']
    splitter_options = ['best', 'random']
    max_features_options = ['auto', 'sqrt', 'log2', None]
    class_weight_options = [None, 'balanced']

    best_criterion = criterion_options[best['criterion']]
    best_splitter = splitter_options[best['splitter']]
    best_max_features = max_features_options[best['max_features']]
    best_class_weight = class_weight_options[best['class_weight']]
    # For max_leaf_nodes, handle None as a separate case
    max_leaf_nodes_options = [None] + list(range(2, 100))
    best_max_leaf_nodes = max_leaf_nodes_options[best['max_leaf_nodes']]

    # For parameters not using `hp.choice`, you can use them directly
    best_max_depth = best['max_depth']
    best_min_samples_split = int(best['min_samples_split'] * 100)
    best_min_samples_leaf = int(best['min_samples_leaf'] * 100)
    best_min_weight_fraction_leaf = best['min_weight_fraction_leaf']

    # Train the Decision Tree classifier with the best parameters
    classifier = DecisionTreeClassifier(
        criterion=best_criterion,
        splitter=best_splitter,
        max_depth=best_max_depth,
        min_samples_split=best_min_samples_split,
        min_samples_leaf=best_min_samples_leaf,
        max_features=best_max_features,
        class_weight=best_class_weight,
        min_weight_fraction_leaf=best_min_weight_fraction_leaf,
        max_leaf_nodes=best_max_leaf_nodes
    )

    # Assuming X and y are your data and labels, and X has been encoded as X_en if necessary
    classifier.fit(X, y)  # Replace X with X_en if your dataset is encoded
    return classifier

preform_dt_parameter_tuning(X_en,y)

100%|████████████████████████████████████████████| 200/200 [00:35<00:00,  5.66trial/s, best loss: -0.37777777777777777]


DecisionTreeClassifier(class_weight='balanced', max_depth=12,
                       max_features='auto', min_samples_leaf=15,
                       min_samples_split=9,
                       min_weight_fraction_leaf=0.051268956971994385,
                       splitter='random')

In [67]:
def perform_knn_parameter_tuning(X, y):
    # Define the search space for KNN parameters
    space = {
        'n_neighbors': hp.choice('n_neighbors', range(1, 31)),  # Number of neighbors to use
        'weights': hp.choice('weights', ['uniform', 'distance']),  # Weight function used in prediction
        'algorithm': hp.choice('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute']),  # Algorithm to compute nearest neighbors
        'leaf_size': hp.choice('leaf_size', range(1, 50)),  # Leaf size passed to BallTree or KDTree
        'p': hp.choice('p', [1, 2]),  # Power parameter for the Minkowski metric
    }

    # Objective function with LOO cross-validation
    def objective(space):
        classifier = KNeighborsClassifier(
            n_neighbors=space['n_neighbors'],
            weights=space['weights'],
            algorithm=space['algorithm'],
            leaf_size=space['leaf_size'],
            p=space['p']
        )

        # Using Leave-One-Out cross-validation
        loo = LeaveOneOut()
        accuracy = cross_val_score(classifier, X, y, cv=loo, scoring='f1', n_jobs=-1).mean()

        # Return the loss and status
        return {'loss': -accuracy, 'status': STATUS_OK}

    # Run the optimization
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=100,  # Adjust the number of evaluations based on computational budget
        trials=trials
    )

    # Map `hp.choice` indices to their corresponding values
    weights_options = ['uniform', 'distance']
    algorithm_options = ['auto', 'ball_tree', 'kd_tree', 'brute']
    p_options = [1, 2]

    best_weights = weights_options[best['weights']]
    best_algorithm = algorithm_options[best['algorithm']]
    best_p = p_options[best['p']]
    
    # For parameters using `hp.choice`, map indices to values
    best_n_neighbors = best['n_neighbors'] + 1  # `range` starts at 1 for `n_neighbors`
    best_leaf_size = best['leaf_size'] + 1  # `range` starts at 1 for `leaf_size`

    # Instantiate the KNN classifier with the best parameters
    classifier = KNeighborsClassifier(
        n_neighbors=best_n_neighbors,
        weights=best_weights,
        algorithm=best_algorithm,
        leaf_size=best_leaf_size,
        p=best_p
    )

    # Train the KNN classifier with the best parameters
    classifier.fit(X, y)
    return classifier
perform_knn_parameter_tuning(X_en,y)

100%|█████████████████████████████████████████████| 100/100 [00:12<00:00,  8.05trial/s, best loss: -0.6222222222222222]


KNeighborsClassifier(algorithm='kd_tree', leaf_size=26, n_neighbors=14)

In [66]:
def perform_lr_parameter_tuning(X, y):
    # Define the search space for logistic regression
    space = {
        'C': hp.loguniform('C', np.log(0.001), np.log(1000)),  # Regularization strength
        'penalty': hp.choice('penalty', ['l1', 'l2', 'elasticnet', 'none']),  # Regularization type
        'solver': hp.choice('solver', ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']),  # Algorithm to use
    }

    # Make sure to handle the combination of penalty and solver correctly
    # Objective function with LOO cross-validation
    def objective(space):
        # Conditional logic to ensure compatibility between solver and penalty
        if space['penalty'] == 'elasticnet' and space['solver'] not in ['saga']:
            return {'loss': 1, 'status': STATUS_OK}  # High loss for incompatible combination
        if space['penalty'] == 'l1' and space['solver'] not in ['liblinear', 'saga']:
            return {'loss': 1, 'status': STATUS_OK}
        
        # Instantiate the logistic regression model
        model = LogisticRegression(
            C=space['C'],
            penalty=space['penalty'],
            solver=space['solver'],
            max_iter=1000  # Ensure convergence
        )

        # Using Leave-One-Out cross-validation
        loo = LeaveOneOut()
        score = cross_val_score(model, X, y, cv=loo, scoring='f1', n_jobs=-1).mean()

        # Return the loss
        return {'loss': -score, 'status': STATUS_OK}

    # Run the optimization
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=100,
        trials=trials
    )

    # Instantiate and fit the model with the best parameters
    best_model = LogisticRegression(
        C=best['C'],
        penalty=['l1', 'l2', 'elasticnet', 'none'][best['penalty']],
        solver=['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'][best['solver']],
        max_iter=1000
    ).fit(X, y)

    # Return the fitted model
    return best_model
perform_lr_parameter_tuning(X_en,y)

100%|█████████████████████████████████████████████| 100/100 [00:11<00:00,  9.03trial/s, best loss: -0.6222222222222222]


LogisticRegression(C=0.012821684098058778, max_iter=1000, penalty='l1',
                   solver='saga')

In [ ]:
def perform_svm_parameter_tuning(X, y):
    # Define the search space for SVM parameters
    space = {
        'C': hp.loguniform('C', -5, 15),  # Penalty parameter C of the error term
        'kernel': hp.choice('kernel', ['rbf', 'sigmoid']),  # Specifies the kernel type to be used in the algorithm
        'gamma': hp.choice('gamma', ['scale', 'auto']),  # Kernel coefficient for 'rbf', 'poly', and 'sigmoid'
        'degree': hp.choice('degree', range(1, 6)),  # Degree of the polynomial kernel function ('poly')
        'coef0': hp.uniform('coef0', 0, 10),  # Independent term in kernel function. It is only significant in 'poly' and 'sigmoid'
    }

    # Objective function with LOO cross-validation
    def objective(space):
        classifier = SVC(
            C=space['C'],
            kernel=space['kernel'],
            gamma=space['gamma'],
            degree=space['degree'],
            coef0=space['coef0'],
            probability=True  # Enable probability estimates (necessary for certain evaluations)
        )

        # Using Leave-One-Out cross-validation
        loo = LeaveOneOut()
        accuracy = cross_val_score(classifier, X, y, cv=loo, scoring='f1', n_jobs=-1).mean()

        # Return the loss and status
        return {'loss': -accuracy, 'status': STATUS_OK}

    # Run the optimization
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=100,  # Adjust based on computational budget
        trials=trials
    )

    # Map `hp.choice` indices to their corresponding values
    kernel_options = ['rbf', 'sigmoid']
    gamma_options = ['scale', 'auto']

    best_kernel = kernel_options[best['kernel']]
    best_gamma = gamma_options[best['gamma']]
    best_degree = best['degree'] + 1  # Adjust if needed

    # Instantiate the SVM classifier with the best parameters
    classifier = SVC(
        C=best['C'],
        kernel=best_kernel,
        gamma=best_gamma,
        degree=best_degree,
        coef0=best['coef0'],
        probability=True  # As mentioned above
    )

    # Train the SVM classifier with the best parameters
    classifier.fit(X, y)
    return classifier
perform_svm_parameter_tuning(X_en,y)

 49%|██████████████████████                       | 49/100 [00:06<00:06,  7.50trial/s, best loss: -0.28888888888888886]